<a href="https://colab.research.google.com/github/maxbobrov85/DataScience/blob/main/homework_1_bobrovmv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

### Нахождение похожих товаров

In [2]:
import pandas as pd
import numpy as np

In [11]:
#Загрузка данных и их краткий обзор
data = pd.read_csv('/sample-data.csv')
data.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


# Новый раздел

### TF-IDF embedding

In [12]:
#Проведем векторизацию стандартным инструментом
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf.fit_transform(data['description'] ).toarray()

In [13]:
tfidf_matrix.shape

(500, 100)

In [14]:
tfidf_matrix

array([[0.13227945, 0.09026108, 0.0856903 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.21605756, 0.23588353, 0.11196924, ..., 0.22075499, 0.        ,
        0.06504209],
       [0.10600198, 0.14466123, 0.        , ..., 0.09025553, 0.        ,
        0.        ],
       [0.11106583, 0.15157187, 0.07194817, ..., 0.09456714, 0.        ,
        0.        ]])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances,euclidean_distances
metric=cosine_similarity
# Вычисляем косинусное расстояние между парами векторов
similarity_matrix_itidf = metric(tfidf_matrix)
similarity_matrix_itidf

array([[1.        , 0.74784451, 0.60195865, ..., 0.52081562, 0.55956073,
        0.5977328 ],
       [0.74784451, 1.        , 0.77799984, ..., 0.41643437, 0.5557526 ,
        0.58484265],
       [0.60195865, 0.77799984, 1.        , ..., 0.42310676, 0.4141116 ,
        0.44532735],
       ...,
       [0.52081562, 0.41643437, 0.42310676, ..., 1.        , 0.39570938,
        0.43626628],
       [0.55956073, 0.5557526 , 0.4141116 , ..., 0.39570938, 1.        ,
        0.89313602],
       [0.5977328 , 0.58484265, 0.44532735, ..., 0.43626628, 0.89313602,
        1.        ]])

Матрица схожести товаров

In [16]:
threshold = 0.6  # Подобраный порог

# Создаём матрицу с бинарными значениями: 1, если косинусное расстояние больше порога, иначе 0
similarity_matrix_binary_ifidf = (similarity_matrix_itidf > threshold).astype(int)
similarity_matrix_binary_ifidf

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1]])

### Word2Vec embedding

In [17]:
#Воспользуемся стандартным инструментом библиотеки gensim
from gensim.models import Word2Vec
import string

def preprocess_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = text.translate(str.maketrans('', '', string.punctuation))  # Удаление знаков пунктуации
    return text

data['cleaned_text'] = data['description'].apply(preprocess_text)

# Токенизация текстов
data['tokenized_text'] = data['cleaned_text'].str.split()

# Обучение модели Word2Vec. Гипре параметры подобраны для лучшего отображения схожести текстов
model = Word2Vec(sentences=data['tokenized_text'], vector_size=150, window=3, min_count=4, sg=1)




In [18]:
def get_average_vector(text, model):
    vector_sum = 0
    word_count = 0
    for word in text:
        if word in model.wv:
            vector_sum += model.wv[word]
            word_count += 1
    if word_count > 0:
        return vector_sum / word_count
    else:
        return None

data['word2vec_vector'] = data['tokenized_text'].apply(lambda x: get_average_vector(x, model))

# Вычисление косинусного расстояния

word2vec_vectors = list(data['word2vec_vector'])
similarity_matrix_w2v =  metric(word2vec_vectors, word2vec_vectors)
similarity_matrix_w2v

array([[1.        , 0.9940897 , 0.99002355, ..., 0.9330103 , 0.9502648 ,
        0.97123975],
       [0.9940897 , 0.9999999 , 0.9910671 , ..., 0.93408185, 0.9643102 ,
        0.9826256 ],
       [0.99002355, 0.9910671 , 1.0000001 , ..., 0.9617422 , 0.9519488 ,
        0.96631575],
       ...,
       [0.93301034, 0.9340825 , 0.9617424 , ..., 1.0000002 , 0.89547384,
        0.9091823 ],
       [0.95026475, 0.9643103 , 0.95194876, ..., 0.8954742 , 1.0000004 ,
        0.99177486],
       [0.97123975, 0.9826256 , 0.96631575, ..., 0.9091823 , 0.99177486,
        1.0000001 ]], dtype=float32)

Матрица схожести товаров

In [19]:
threshold = 0.99  # Подобранный порог

# Создаём матрицу с бинарными значениями: 1, если косинусное расстояние меньше порога, иначе 0
similarity_matrix_binary_w2w = (similarity_matrix_w2v > threshold).astype(int)
similarity_matrix_binary_w2w

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1]])

### Вывод num наиболее похожих товаров

In [20]:
#Функция выбора num (по умолчанию 3) самых близких элементов
def close_topic(topic_num, matrix, num=3):
    close_topics=sorted(enumerate(matrix[topic_num]), key=lambda x:x[1], reverse=True)
    top_indices = [index for index, _ in close_topics[1:num+1]]
    return top_indices

In [21]:
topic_num=0 #Номер товара для сравнения
num=5 # Количество наиболее похожих товаров


Для TF-IDF

In [22]:
indices=close_topic(topic_num, similarity_matrix_itidf, num = num)
print(data['description'][topic_num])
print(data['description'][indices])


Active classic boxers - There's a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><br>

Для Word2Vec

In [23]:
indices=close_topic(topic_num,similarity_matrix_w2v,num=5)
print(data['description'][topic_num])
print(data['description'][indices])

Active classic boxers - There's a reason why our boxers are a cult favorite - they keep their cool, especially in sticky situations. The quick-drying, lightweight underwear takes up minimal space in a travel pack. An exposed, brushed waistband offers next-to-skin softness, five-panel construction with a traditional boxer back for a classic fit, and a functional fly. Made of 3.7-oz 100% recycled polyester with moisture-wicking performance. Inseam (size M) is 4 1/2". Recyclable through the Common Threads Recycling Program.<br><br><b>Details:</b><ul> <li>"Silky Capilene 1 fabric is ultralight, breathable and quick-to-dry"</li> <li>"Exposed, brushed elastic waistband for comfort"</li> <li>5-panel construction with traditional boxer back</li> <li>"Inseam (size M) is 4 1/2"""</li></ul><br><br><b>Fabric: </b>3.7-oz 100% all-recycled polyester with Gladiodor natural odor control for the garment. Recyclable through the Common Threads Recycling Program<br><br><b>Weight: </b>99 g (3.5 oz)<br><br>